# Описание проекта:

**Мотивация:** Среди моих знакомых и родных в последнее время часто идут разговоры о недвижимости в москве,  у кого-то она есть, а кто-то планирует переезжать да и я сам хочу пресмотреть что-нибудь в МО. Поэтому решил разобраться в этой теме и используя отрытые данные построить модель оценки стоимости домов и таунхаусов (позже возможно добавлю квартиры).

**Цель:** Создать модель машинного обучения для предсказания стоимости недвижимости (дома и таунхаусы) в Москве и подмосковье с использованием данных, полученных с помощью библиотеки Cianparser.

**Целевая аудитория:**

* Жители Москвы и московской области, владеющие недвижимостью и желающие оценить её стоимость.
* Люди, планирующие переезд в Раменское или Жуковский и интересующиеся ценами на недвижимость.

**Инструменты:**

* Python
* Библиотека Cianparser для сбора данных
* Библиотеки машинного обучения (Scikit-learn)
* Streamlit для создания интерактивного приложения

**Ожидаемые результаты:**

* Модель машинного обучения с MAPE <= 0.2, способная точно предсказывать стоимость недвижимости.
* Интерактивное приложение Streamlit, позволяющее пользователям вводить характеристики недвижимости и получать прогноз цены.

## Парсинг данных

In [273]:
import time
from tqdm import tqdm
import chime
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup
import cianparser
import requests
import re

chime.theme('mario')
%load_ext chime

The chime extension is already loaded. To reload it, use:
  %reload_ext chime


In [274]:
locs = pd.DataFrame(cianparser.list_locations()) 
locs.columns = ['Город','Индекс']

In [275]:
locs[locs['Город'] == 'Красногорск']

,Город,Индекс
126,Красногорск,175071


In [276]:
locations = ['Жуковский','Раменское','Одинцово','Домодедово',
            'Реутов','Железнодорожный','Долгопрудный','Москва',
            'Красногорск','Подольск','Балашиха']

In [279]:
def parce_suburban(city, suburban_types = ["house",'townhouse'], pages_ranges = [[1,1],[1,1]],
                   parsed_pages=0,return_counter=False, old_df=pd.read_csv('suburban.csv')):
    
    """
    Парсит данные о загородной недвижимости с сайта Циан для заданного города и типов недвижимости.

    Args:
        city (str): Название города для парсинга.
        suburban_types (list): Список типов загородной недвижимости (например, "house", "townhouse").
        pages_ranges (list): Список диапазонов страниц для парсинга для каждого типа недвижимости.
        parsed_pages (int): Количество уже спарсенных страниц (для управления таймаутами).
        return_counter (bool): Если True, возвращает также количество спарсенных страниц.
        old_df (pd.DataFrame): DataFrame с ранее спарсенными данными для проверки на дубликаты.

    Returns:
        pd.DataFrame or tuple: DataFrame с новыми данными о загородной недвижимости или кортеж 
                               (DataFrame, количество спарсенных страниц), если return_counter=True.
    """
    
    parser = cianparser.CianParser(location=city)
    df = pd.DataFrame()
    data = []

    for suburban_type,pages_range in zip(suburban_types,pages_ranges):
        for page in range(pages_range[0], pages_range[1]+1):
            
            print(f'-----------------------------')
            print(f'Локация: {city}')
            print(f'Вид недвижимости: {suburban_type}')
            print()
            
            # Проверка есть-ли новые ссылки
            fast_suburban_sale = pd.DataFrame(parser.get_suburban( 
                                                deal_type="sale",
                                                suburban_type = suburban_type,
                                                additional_settings={"start_page":page,
                                                                    "end_page":page}
                                                ))
            if fast_suburban_sale.shape[0] > 0:
            
                ids = [x not in old_df['url'].to_list() for x in fast_suburban_sale['url'].to_list()]
                new_df = fast_suburban_sale[ids]
                new_size = new_df.shape[0]

                # Если есть, то скачивается страница с новыми данными
                if new_size > 0:
                    print(f'-----------------------------')
                    print(f'Найдено новых обьявлений: {new_size}')
                    print(f'-----------------------------')
                    if parsed_pages >= 2:
                        num = np.random.randint(100,120)
                        print(f'parsed pages: {parsed_pages}')
                        print(f'timeout: {num}')
                        time.sleep(num)
                        
                    suburban_sale = pd.DataFrame(parser.get_suburban( 
                                                        deal_type="sale",
                                                        suburban_type = suburban_type,
                                                        with_extra_data=True,
                                                        additional_settings={"start_page":page,
                                                                            "end_page":page}
                                                        ))
                    ids = [x not in old_df['url'].to_list() for x in suburban_sale['url'].to_list()]
                    good_sales = suburban_sale[ids]
                    
                    data.append(good_sales)
                    parsed_pages +=1
                else:
                    print(f'-----------------------------')
                    print(f'Новых обьявлений не найдено')
                    print(f'-----------------------------')
                    pass
            else:
                print(f'-----------------------------')
                print(f'Новых обьявлений не найдено')
                print(f'-----------------------------')
                pass

    # Список словарей из полученных данных преобразется в датафрейм

    if len(data) > 0:
        df = pd.concat(data).drop_duplicates().reset_index(drop=True)
    
    if return_counter:
        return df,parsed_pages
    else:
        return df
  

In [319]:
def create_address(row, reverse_order=False, drop_district=False, with_mo = True):
    """
    Формирует полный адрес объекта недвижимости из отдельных столбцов DataFrame.

    Args:
        row (pd.Series): Строка DataFrame, содержащая информацию об адресе.
        reverse_order (bool): Если True, формирует адрес в обратном порядке.

    Returns:
        str: Полный адрес объекта недвижимости.
    """
    country = 'Россия, '

    if with_mo:
        MO = 'Московская область, '
    else:
        MO = ''

    if reverse_order:
        if drop_district:
            address = country + MO + f"{row['location']}, {row['street']}, {row['house_number']}" 
        else:
            address = country + MO + f"{row['location']}, {row['district']}, {row['street']}, {row['house_number']}" 
    else:
        if drop_district:
            address = country + MO + f"{row['street']}, {row['house_number']}, {row['location']}" 
        else:
            address = country + MO + f"{row['street']}, {row['house_number']}, {row['district']}, {row['location']}"

    address = address.strip().replace(", , ,", ",").replace(", ,", ",")
    if address.startswith(","):
        address = address[1:]
    return address

geolocator = Nominatim(user_agent="my_app", timeout=60)

In [320]:
def geocode_address(row):
    """
    Геокодирует адрес с помощью OpenStreetMap API, пробуя различные варианты адреса.

    Args:
        row (pd.Series): Строка DataFrame, содержащая информацию об адресе.

    Returns:
        tuple: Координаты (latitude, longitude) объекта, если геокодирование успешно, иначе None.
    """
    address_options = [
        create_address(row.fillna('')),
        create_address(row.fillna(''), reverse_order=True),
        create_address(row.fillna(''), drop_district=True),
        create_address(row.fillna(''), reverse_order=True, drop_district=True),
        create_address(row.fillna(''), with_mo=False),
        create_address(row.fillna(''), reverse_order=True, with_mo=False),
        create_address(row.fillna(''), drop_district=True, with_mo=False),
        create_address(row.fillna(''), reverse_order=True, drop_district=True, with_mo=False),
    ]

    for address in address_options:
        try:
            location = geolocator.geocode(address)
            return (location.latitude, location.longitude)
        except:
            try:
                location = geolocator.geocode(address).replace('-я','')
                return (location.latitude, location.longitude)
            except:
                pass  # Продолжаем пробовать другие варианты адреса

    return None  # Если ни один вариант не сработал

In [281]:
%%time
%%chime
dfs = []
old_df = pd.read_csv('suburban.csv')
counter = 0

for location in locations:
    
    df,counter = parce_suburban(location, parsed_pages = counter, return_counter = True,
                                old_df=old_df)
    if not df.empty:
        dfs.append(df)

-----------------------------
Локация: Жуковский
Вид недвижимости: house


                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=4750&deal_type=sale&offer_type=suburban&object_type%5B0%5D=1

 1 | 1 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 28. Progress ratio: 100 %. Average price: 36 375 347 rub

The collection of information from the pages with list of offers is completed
Total number of parsed offers: 28. 
-----------------------------
Найдено новых обьявлений: 1
-----------------------------

                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=4750&deal_type=sale&offer_type=suburban&object_type%5B0%5D=1

 1 | 1 page with lis

In [282]:
df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
df['date'] = dt.date.today()
df

,author,author_type,url,location,deal_type,accommodation_type,suburban_type,price_per_month,commissions,price,...,sewage_system,bathroom,living_meters,floors_count,phone,district,underground,street,house_number,date
0,Центральное,real_estate_agent,https://zhukovsky.cian.ru/sale/suburban/295148...,Жуковский,sale,suburban,house,-1,0,4595000,...,-1,-1,44 м²,2,+79057159228,,м. Кратово,,,2024-04-26
1,МИРА - городская недвижимость,real_estate_agent,https://ramenskoye.cian.ru/sale/suburban/29378...,Раменское,sale,suburban,townhouse,-1,0,15500000,...,-1,-1,130 м²,2,+79153499147,,,улица 2-я Осенняя,95,2024-04-26
2,АЛЬТЕРНАТИВА,real_estate_agent,https://ramenskoye.cian.ru/sale/suburban/29275...,Раменское,sale,suburban,townhouse,-1,0,11000000,...,-1,-1,"124,9 м²",1,+79166742577,,,улица Гвардейская,,2024-04-26
3,ID 13118779,unknown,https://ramenskoye.cian.ru/sale/suburban/29313...,Раменское,sale,suburban,townhouse,-1,0,11200000,...,-1,-1,"156,6 м²",2,+79856338236,,,улица Майская,5,2024-04-26
4,ID 63989286,real_estate_agent,https://ramenskoye.cian.ru/sale/suburban/29846...,Раменское,sale,suburban,townhouse,-1,0,11450000,...,-1,-1,271 м²,2,+79895774735,,,улица Красная,94,2024-04-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,ID 56594583,homeowner,https://chekhov.cian.ru/sale/suburban/292848690/,Подольск,sale,suburban,townhouse,-1,0,5200000,...,-1,-1,50 м²,-1,+79637256787,,,Лесная улица,,2024-04-26
58,Landis,real_estate_agent,https://podolsk.cian.ru/sale/suburban/296187870/,Подольск,sale,suburban,townhouse,-1,0,10900000,...,-1,-1,120 м²,2,+79660619733,,,,,2024-04-26
59,KASKAD Недвижимость,developer,https://balashikha.cian.ru/sale/suburban/29988...,Балашиха,sale,suburban,house,-1,0,59300000,...,-1,-1,240 м²,2,+79057493007,,,Аптекарская улица,52,2024-04-26
60,KASKAD Недвижимость,developer,https://balashikha.cian.ru/sale/suburban/29988...,Балашиха,sale,suburban,house,-1,0,39375000,...,-1,-1,240 м²,2,+79057493007,,,Аптекарская улица,54,2024-04-26


In [283]:
%%chime

new_df = df[[x not in old_df['url'].to_list() for x in df['url'].to_list()]]
new_size = new_df.shape[0]
display(new_df)

,author,author_type,url,location,deal_type,accommodation_type,suburban_type,price_per_month,commissions,price,...,sewage_system,bathroom,living_meters,floors_count,phone,district,underground,street,house_number,date
0,Центральное,real_estate_agent,https://zhukovsky.cian.ru/sale/suburban/295148...,Жуковский,sale,suburban,house,-1,0,4595000,...,-1,-1,44 м²,2,+79057159228,,м. Кратово,,,2024-04-26
1,МИРА - городская недвижимость,real_estate_agent,https://ramenskoye.cian.ru/sale/suburban/29378...,Раменское,sale,suburban,townhouse,-1,0,15500000,...,-1,-1,130 м²,2,+79153499147,,,улица 2-я Осенняя,95,2024-04-26
2,АЛЬТЕРНАТИВА,real_estate_agent,https://ramenskoye.cian.ru/sale/suburban/29275...,Раменское,sale,suburban,townhouse,-1,0,11000000,...,-1,-1,"124,9 м²",1,+79166742577,,,улица Гвардейская,,2024-04-26
3,ID 13118779,unknown,https://ramenskoye.cian.ru/sale/suburban/29313...,Раменское,sale,suburban,townhouse,-1,0,11200000,...,-1,-1,"156,6 м²",2,+79856338236,,,улица Майская,5,2024-04-26
4,ID 63989286,real_estate_agent,https://ramenskoye.cian.ru/sale/suburban/29846...,Раменское,sale,suburban,townhouse,-1,0,11450000,...,-1,-1,271 м²,2,+79895774735,,,улица Красная,94,2024-04-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,ID 56594583,homeowner,https://chekhov.cian.ru/sale/suburban/292848690/,Подольск,sale,suburban,townhouse,-1,0,5200000,...,-1,-1,50 м²,-1,+79637256787,,,Лесная улица,,2024-04-26
58,Landis,real_estate_agent,https://podolsk.cian.ru/sale/suburban/296187870/,Подольск,sale,suburban,townhouse,-1,0,10900000,...,-1,-1,120 м²,2,+79660619733,,,,,2024-04-26
59,KASKAD Недвижимость,developer,https://balashikha.cian.ru/sale/suburban/29988...,Балашиха,sale,suburban,house,-1,0,59300000,...,-1,-1,240 м²,2,+79057493007,,,Аптекарская улица,52,2024-04-26
60,KASKAD Недвижимость,developer,https://balashikha.cian.ru/sale/suburban/29988...,Балашиха,sale,suburban,house,-1,0,39375000,...,-1,-1,240 м²,2,+79057493007,,,Аптекарская улица,54,2024-04-26


In [328]:
%%time
%%chime

if new_df.shape[0] > 0:
    new_df['address'] = new_df[geo_cols].fillna('').apply(create_address, axis=1)

    coordinates = []
    for index, row in tqdm(new_df.iterrows(), total=new_df.shape[0]):
        coordinates.append(geocode_address(row))

    new_df['coordinates'] = coordinates

    num_missing_coords = new_df['coordinates'].isna().sum()
    num_geocoded = len(new_df) - num_missing_coords
    print(f"Количество объектов с координатами: {num_geocoded}")
    print(f"Количество объектов без координат: {num_missing_coords}")

    full_df = (pd.concat([old_df,new_df])
            .drop_duplicates(subset=['url','author'],ignore_index=True)
            .reset_index(drop=True))

    full_df.to_csv('suburban.csv',index=False)

    
    print(f'Количество новых обьялений: {new_size}')

else:
    full_df = old_df
    print('Новых обьявлений не обнаруженно')

print(f'Суммарное количество объялений: {full_df.shape[0]}')


  0%|          | 0/62 [00:00<?, ?it/s]

100%|██████████| 62/62 [03:56<00:00,  3.82s/it]

Количество объектов с координатами: 52
Количество объектов без координат: 10
Количество новых обьялений: 62
Суммарное количество объялений: 2948
CPU times: total: 1.25 s
Wall time: 3min 56s
